## Before doing anything with this dataset...

1. What fields are missing?

- Quantity
- is_active

2. Are there duplicates in dataset?

Potentially yes, because products are scraped from different collections. For examples, 'New arrivals' and 'Womens' have many products in common.

3. How to sort out duplicates?

Title

4. How to organize data?

The goals is to keep collections and process data without duplicates
- Create a new dataset of all products
- Add collections as a new field

5. If we only data analytics, do description, material & care matter?

Generally no, but if we use semantics analytics, then they matter. It's better to keep everything, but not in the dataset we currently work on

6. What is our plan?

- Filter out description, product details, material & care
- Add missing fields and generate data
- Create a new dataset for all products
- Add collections as a new field

7. Actual plan

- Filter out description, product details, material & care
- Filter duplicates by item titles and save collections for each item
- Create a new dataset for all products
- Convert types
- Decouple colors and sizes 
- Add missing fields (quantity, is_active) and generate data
- Save dataset

In [83]:
import numpy as np
import pandas as pd
from collections import defaultdict
from itertools import product

In [84]:
df = pd.read_csv('../webscrape/data/products.csv')
df.columns
df

,Collection,Title,Current price,Colors,Sizes,Description,Product details,Materials & care
0,public-goods-food,Sesame Paste Ramen 5 Pack,$12.90,[],[],Your favorite snack just got a lot healthier w...,[],[]
1,public-goods-food,Organic Linden Flower Raw Honey,$9.90,[],[],Sweeten your tea or açaí bowl with this 100% o...,[],[]
2,public-goods-food,Seaweed Snacks,$9.90,[],[],Indulge in a healthy and flavorful snack with ...,[],[]
3,public-goods-food,Veggie Chips,$6.90,[],[],"These thin, crispy veggie chips are an uncommo...",[],[]
4,public-goods-food,Organic Wildflower Raw Honey,$18.90,[],[],100% organic wildflower honey is harvested fro...,[],[]
...,...,...,...,...,...,...,...,...
4826,denim-guide-sustainable-jeans,Men's Denim Regular Pants Dark Navy (L 30inch ...,$49.90,['Dark Navy'],"['29', '30', '31', '32', '33', '34', '35', '36']","Made with organic cotton. We use an undyed, na...","['<strong>Style Code: </strong>AEK6423A', '<st...","['<span class=""metafield-multi_line_text_field..."
4827,denim-guide-sustainable-jeans,Men's Denim Wide Pants Dark Navy (30 Inch / 76...,$49.90,['Dark Navy'],"['29', '30', '31', '32', '33', '34', '35', '36']",Cotton denim pants that fit better the more yo...,"['<strong>Style Code: </strong>AE0W023A', '<st...","['<span class=""metafield-multi_line_text_field..."
4828,denim-guide-sustainable-jeans,Men's Denim Wide Pants Blue (30 Inch / 76 cm),$49.90,['Blue'],"['29', '30', '31', '32', '33', '34', '35', '36']",Light colored look achieved by reducing the dy...,"['<strong>Style Code: </strong>AE0VZ23A', '<st...","['<span class=""metafield-multi_line_text_field..."
4829,denim-guide-sustainable-jeans,Men's Denim Wide Pants Dark Navy (30 Inch / 76...,$49.90,['Dark Navy'],"['28', '29', '30', '31', '32', '33', '34', '35']",Cotton denim pants that fit better the more yo...,"['<strong>Style Code: </strong>AE0W024S', '<st...","['<span class=""metafield-multi_line_text_field..."


In [85]:
# Filter out description, product details, material & care
data = []
for i, row in df.iterrows():
    data.append(row[:5])

new_df = pd.DataFrame(data)
new_df.info
new_df.columns

Index(['Collection', 'Title', 'Current price', 'Colors', 'Sizes'], dtype='object')

In [86]:
# Filter duplicates by item titles and save collections for each item
item_map = defaultdict(list)
for i, row in new_df.iterrows():
    item_map[row.loc['Title']].append(row.loc['Collection'])
print(len(item_map.keys()))
print(item_map["Men's Non-Iron Button Down Shirt"])


1561
['mens-shirts-polos', 'mens-shirts-polos', 'new-arrivals', 'mens', 'mens', 'mens-tops', 'mens-tops', 'workwear']


In [87]:
# Create a new dataset for all products
data = []
visited = set()
for i, row in new_df.iterrows():
    title = row.loc['Title']
    if not title: print(title)
    if title not in visited:
        price = row.loc['Current price']
        colors = row.loc['Colors']
        sizes = row.loc['Sizes']
        data.append([title, item_map[title], price, colors, sizes])
        visited.add(title)

filtered_df = pd.DataFrame(data, columns=['title', 'collections', 'current_price', 'colors', 'sizes'])
filtered_df

,title,collections,current_price,colors,sizes
0,Sesame Paste Ramen 5 Pack,"[public-goods-food, food, public-goods, dry-so...",$12.90,[],[]
1,Organic Linden Flower Raw Honey,"[public-goods-food, food, kitchen-dining, publ...",$9.90,[],[]
2,Seaweed Snacks,"[public-goods-food, food, public-goods, home-e...",$9.90,[],[]
3,Veggie Chips,"[public-goods-food, food, public-goods]",$6.90,[],[]
4,Organic Wildflower Raw Honey,"[public-goods-food, home-essentials-new-arrivals]",$18.90,[],[]
...,...,...,...,...,...
1556,Cord Hair Band 2 Pieces Set,"[hair-care, makeup]",$4.90,[],[]
1557,"Pine Shelf Unit - Cross Bar - Large (33.9"")",[pine-shelving-unit],$15.00,[],[]
1558,SUS Shelving Unit - Walnut - Regular - Medium,[home-essentials-new-arrivals],$350.00,[],[]
1559,Heatproof Glass Pot - 25.3 oz,[glassware],$29.90,[],[]


In [88]:
# Convert current_price type to float
filtered_df['current_price'] = filtered_df['current_price'].str.replace('$', '').str.replace(',', '')
filtered_df = filtered_df.astype({'current_price': 'float64'})

# Convert colors and sizes type to list
filtered_df['colors'] = filtered_df['colors'].replace(r'[\s+\[\]\'+]', '', regex=True)
filtered_df['colors'] = filtered_df['colors'].str.split(',')
filtered_df['sizes'] = filtered_df['sizes'].replace(r'[\s+\[\]\'+]', '', regex=True)
filtered_df['sizes'] = filtered_df['sizes'].str.split(',')
filtered_df

,title,collections,current_price,colors,sizes
0,Sesame Paste Ramen 5 Pack,"[public-goods-food, food, public-goods, dry-so...",12.9,[],[]
1,Organic Linden Flower Raw Honey,"[public-goods-food, food, kitchen-dining, publ...",9.9,[],[]
2,Seaweed Snacks,"[public-goods-food, food, public-goods, home-e...",9.9,[],[]
3,Veggie Chips,"[public-goods-food, food, public-goods]",6.9,[],[]
4,Organic Wildflower Raw Honey,"[public-goods-food, home-essentials-new-arrivals]",18.9,[],[]
...,...,...,...,...,...
1556,Cord Hair Band 2 Pieces Set,"[hair-care, makeup]",4.9,[],[]
1557,"Pine Shelf Unit - Cross Bar - Large (33.9"")",[pine-shelving-unit],15.0,[],[]
1558,SUS Shelving Unit - Walnut - Regular - Medium,[home-essentials-new-arrivals],350.0,[],[]
1559,Heatproof Glass Pot - 25.3 oz,[glassware],29.9,[],[]


- Create collection, color and size table
- Map relationships
- Add missing fields (quantity, is_active) and generate data

In [89]:
collection_set = set()
attributes = set()
belongs = {'product_id': [], 'collection': []}
product_attr = {'product_id': [], 'color': [], 'size': []}

for i, row in filtered_df.iterrows():
    # if i < 18: continue
    # if i > 19: break
    title = row.loc['title']
    collections = row.loc['collections']
    colors = row.loc['colors']
    sizes = row.loc['sizes']

    for c in collections:
        # append to relationship table
        belongs['product_id'].append(i)
        belongs['collection'].append(c)

        # append to collections data
        if c not in collection_set:
            collection_set.add(c)


    if type(colors) == list and type(sizes) == list:
        # append to relationship table
        attr_set = list(product(colors, sizes))
        for color, size in attr_set:
            product_attr['product_id'].append(i)
            product_attr['color'].append(color if color else None)
            product_attr['size'].append(size if size else None)

            # append to attribute data
            if (color, size) not in attributes:
                attributes.add((color, size))

collection_df = pd.DataFrame(list(collection_set), columns=['name'])
attribute_df = pd.DataFrame(list(attributes), columns=['color', 'size'])
belongs_df = pd.DataFrame(belongs)
product_attr_df = pd.DataFrame(product_attr).dropna(thresh=2)

product_attr_df

,product_id,color,size
17,17,None,1.1L(1.2Quarts)
18,17,None,2.6L(2.7Quarts)
19,18,OffWhite,"Standard(W16.9xL24.8"")"
20,18,OffWhite,"Queen(W19.7xL27.6"")"
21,18,LightGray,"Standard(W16.9xL24.8"")"
...,...,...,...
12345,1560,DarkGray,S-M
12346,1560,DarkGray,L-XL
12347,1560,GrayishBrown,XXS-XS
12348,1560,GrayishBrown,S-M


In [90]:
# Add missing fields (quantity, is_active) and generate data

# Quantity logic: based on price range
# < $10: 200 items
# $10 - $100: 300 items
# > $100: 50 items
product_df = filtered_df.drop(columns=['collections', 'colors', 'sizes'])

product_df['is_active'] = True
product_attr_df['is_active'] = True

def generate_quantity(id):
    attr_count = product_attr_df[product_attr_df.product_id == id].shape[0]
    price = product_df.loc[id, 'current_price']
    attr_count = 1 if attr_count == 0 else attr_count
    if price <= 10: 
        return 200 // attr_count
    elif 10 < price <= 100: 
        return 300 // attr_count
    else: 
        return 50

product_attr_df['quantity'] = product_attr_df.apply(
    lambda row: generate_quantity(row.loc['product_id']), 
    axis=1
)

product_attr_df

,product_id,color,size,is_active,quantity
17,17,None,1.1L(1.2Quarts),True,150
18,17,None,2.6L(2.7Quarts),True,150
19,18,OffWhite,"Standard(W16.9xL24.8"")",True,37
20,18,OffWhite,"Queen(W19.7xL27.6"")",True,37
21,18,LightGray,"Standard(W16.9xL24.8"")",True,37
...,...,...,...,...,...
12345,1560,DarkGray,S-M,True,33
12346,1560,DarkGray,L-XL,True,33
12347,1560,GrayishBrown,XXS-XS,True,33
12348,1560,GrayishBrown,S-M,True,33


In [92]:
def count_total(id):
    return product_attr_df[product_attr_df.product_id == id].quantity.sum()
    
product_df['total_quantity'] = product_attr_df.apply(
    lambda row: count_total(row.loc['product_id']), 
    axis=1
)
product_df['total_quantity'] = product_df.apply(
    lambda row: generate_quantity(row.name) if np.isnan(row.loc['total_quantity']) else row.loc['total_quantity'], 
    axis=1
)
product_df['title'] = product_df['title'].astype(str)
product_df['total_quantity'] = product_df['total_quantity'].astype(int)

product_df

,title,current_price,is_active,total_quantity
0,Sesame Paste Ramen 5 Pack,12.9,True,300
1,Organic Linden Flower Raw Honey,9.9,True,200
2,Seaweed Snacks,9.9,True,200
3,Veggie Chips,6.9,True,200
4,Organic Wildflower Raw Honey,18.9,True,300
...,...,...,...,...
1556,Cord Hair Band 2 Pieces Set,4.9,True,288
1557,"Pine Shelf Unit - Cross Bar - Large (33.9"")",15.0,True,288
1558,SUS Shelving Unit - Walnut - Regular - Medium,350.0,True,288
1559,Heatproof Glass Pot - 25.3 oz,29.9,True,288


In [94]:
product_df.to_csv('product.csv', index_label="id")

collection_df.to_csv('collection.csv', index_label="id")
attribute_df.to_csv('attribute.csv', index_label="id")

belongs_df.to_csv('belongs.csv')
product_attr_df.to_csv('product_attr.csv')